# UC3M Machine Learning Challenge 2022

**Participant: Miguel Zabaleta Sarasa (100463947) - Alias: "naive ftw"**



In this report, I will describe the pipeline that got me the **second place** in the competition.

Since in reality, this pipeline is not very complex, I will also describe the strategies and models where I did more preprocessing (even though they didn't achieve the best performance in the competition). 

## Part I. Initial approach

The first step I took on this project is to do some basic preliminarly **exploratory data analysis** (EDA).

I made note of the following remarks:

- imbalanced dataset: 67.2% of False class, 32.8% of True
- train set has 3250 observations, and 37 variables (including target and ID variable, so 35 features to predict with)
- test set has 1750 observations, which is quite large considering the size of the training set
- 3157/3250 observations have NaN values, and every column has around 300 missing values, so this has to be addressed
- some variables seem better than other at predicting the target variable **on their own**, which is an indication that they would be good features to keep in the training of the model. This was evaluated based on the density function per target value.

Such features are the following:

features 2,7,10,12,17,24,27,28,30,31,32 **this set of features would be later used as feature selection set**

Here are some of these plots:




![Feature 2 density plot](feat2.jpg)

![Feature 7 density plot](feat7.jpg)

![Feature 10 density plot](feat10.jpg)

The second step I decided to take is to simply try a variety of advanced models without any preprocessing, just to see how well I can do initially.

Surprisingly, this ended up being my best model (which is why I decided to also include some other preprocessing techniques and strategies I studied).

Here, I first tried tuning the hyperparameters of the **HistGradientBoosting** classifier, using the **optuna** framework and submitting the final tuned model to the competition. This model achieved a score of **0.854** in the competition.  

I also tried submitting many different default classifiers to compare the performance of a default model with a tuned one, and see if it changed much (using the mean as missing data imputer).

The classifiers I tested were the following (11 estimators):

LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis,
AdaBoost, 

Bagging, 
Extra Trees,
Gradient Boosting, 
Random Forest, 
HistGradient Boosting,

Gaussian Process,
Logistic Regression,
NuSVC.

<br>

Then, I made an ensemble of these default models using the **VotingClassifier()** method in sklearn.

I submitted both using "soft-voting" and "hard-voting". Soft-voting which means that the class label is predicted based on the maximum value of the sums of predicted probabilities, whereas hard-voting uses majority rule voting.

On the hard-voting estimtor, I tried various combinations of **weights** for each model in the ensemble and tested the performance on a separate test set (from the train dataset). I submitted the best weight combination I could find.

The **best performing model** on the competition was the **ensemble of default models, using soft-voting** (and almost no preprocessing, only the **imputation of NaN values** with the **mean** and the **simple scaler**).

The **soft-voting** model achieved a public score of **0.85894**, while the **hard-voting** model achieved **0.85833**.





## Part II. Developing a strategy



The second part of my implementations involved trying to come up with the **best set of parameters** and **preprocessing steps** possible.

In particular, I wanted the final model to be an ensemble of other models, since this family of models had achieved the best results in the competition so far.

Considering this was done without any preprocessing, one would think that trying a more fined-tuned and thoughtful family of these models would be able to achieve even greater results, based on appropriate preprocessing techniques.

These are the **questions** I tried to study:

- Which are the best 4 or 5 tuned models to include in the ensemble?
  - Which is the best number of folds?
  - Which is the best test size?
- Which is the best imputer for missing data?
- Should we remove some of the observations based on their number of missing values?
- Which is the best scaler?
- Should we fix the imbalance on the dataset?
- Which is the best feature selection?
- Which is the best dimensionality reduction technique or combination of them?




### 1. Selection and tuning of the models

**Best number of folds for Cross Validation**

For a fixed test size, I trained and tuned 5 AdaBoostClassifiers (same parameter space, different seeds) and evaluated their performance on the validation folds, and on a separate test set.

I compared these results using 3 and 5 folds.

In this case, it seemed like 3 folds was a better option as it generally tended to performed better on the test set. This indicates that using 3 folds is a better choice for tuning a model that generalises better on a test set which hasn't been trained on.

This is why I decided to use 3 folds throughout the project.

<br>

**Best test size for Cross Validation**

I trained AdaBoost and Bagging classifiers for 3 different test sizes (15%, 25%, 35%), evaluating their test score and score in the competition.

The reasoning behind this was to study a reasonable test size that could be reliable with the scores in the competition.

This means that the score on the test set would be indicative enough of the score achieved in the competition (better test score, better competition score).

The most reliable test size was **35%**

<br>

**Best tuned models to include in the ensemble**

For these 9 models (AdaBoost, Bagging, Extra Trees, Gradient Boosting, Random Forest, Hist Gradient Boosting, Gaussian Process, NuSVC, XGBoost) I tuned their parameters and submitted them to the competition.

For the tuning, I first tried a small budget of iterations (around 15) using **skopt** and **BayesSearchCV**.

Once I saw which hyperparameter values seem to be best for each model, I constrained the search space accordingly and increased the budget, so as to get a better performance.

The best performing models in the competition where AdaBoost, Hist Gradient Boosting, Gradient Boosting, Random Forest and XGBoost.

These are the models to include in the ensemble in future submissions.

### 2. Preprocessing pipeline

For the preprocessing analysis, I decided I wanted to evaluate the following strategies, based on the previous stated questions.

- **missing value imputation:** mean, median, most frequent, iterative imputer, KNN imputer (and adding a flag feature or not on each imputer) (10 possible options)
- **scalers:** standard, min-max, robust, powertransformer (4 possible options)
- **feature selection:** all features, based on importance (tree-based estimators), EDA (3 possible options)
- **dimensionality reduction:** linear (PCA, SVD, KMeans), nonlinear (KPCA) (4 single options, 3 combinations, 7 total options)


**total number of combinations:** 10x4x3x7 = 840

<br>

It is clearly unfeasible to test all of these preprocessing pipelines.

Therefore, the strategy I wanted to implement is to first analyze on its own which is the best mising value imputer; once selected, analyze the best scaler; and finally, analyze the best feature selection + dimension reduction strategy (after scaling+imputing).

Once all these strategies were tested using oversampling and a simple Random Forest estimator with default values (and multiple seeds when convenient), I would select the top strategies, and train the ensemble of models I previously selected on these various processed datasets.

<br>

I was aware that the performance of a Random Forest estimator may vary compared to the ensemble, because it may happen that another preprocessing pipeline is more suitable for this ensemble.

However, I thought this would be a good strategy to **estimate the performance** of all these different pipelines, provided the limited time and computational resources I had available.

**Best missing value imputation**

I first looked at the number of rows in the training set grouped by the number of missing values.

This is the code and output:



![missing value counts](missing1.jpg)

Given the low number of observations with 8 or more missing values, I decided to test if removing these observations resulted in better predictions.

The performance evaluated over 10 different default Random Forest estimators improved, and it also improved when removing the observations with 7 missing values (removing them from the training set and evaluating it on a test set).

Therefore, I decided to **remove all observations with 7 or more missing values** from this point of the study.

<br>

Next, I evaluated 10 RF models on a test set, differentiating between the 5 imputers and also including a flag indicator or not.

The best performance was provided by the **Iterative imputer with indicator** (with an average of 0.841 F1 score).

<br>

**Best scaler**

Now, removing the mentioned observations and using the Iterative imputer with indicator, I evaluated the performance of the 3 scalers over a test set, based on 10 different Random Forest estimators.

The **results were not improved**. In any case, the best scaler was the **robust**, with an 0.822 average F1 score.

<br>

**Best feature selection + dimensionality reduction**
For the feature selection, I first had to evaluate which where the most consistent variables selected by the importance value, among the models used in the ensemble. In this case, the models that had the attribute **feature_importances** were AdaBoost, Gradient Boosting and Random Forest.

The following are the images and code needed to compute the mean decrease in impurity after removing each variable.

![importances code](importances1.jpg)

![importances plots](importances2.jpg)

According to the results, I made the following observations:

**AdaBoost:** Doesn't give much importance to any particular set of variables, only feature 16 is reasonable larger than the others.

**Gradient Boosting:** Gives much more importance to features 2,10,12,28,31,32

**Random Forest:** Gives much more importance to features 2,5,7,10,12,28,31,32

**Final feature selection:** Features 2,5,7,10,12,16,28,31,32 (9 variables)

The next step was to evaluate every dimension reduction combination for all variables, importance variables, and EDA variables (and the previously selected preprocessing steps).

This is a summary of the F1 score I obtained testing one default Random Forest model on a test set of 35% of the training set.

- **no dimension reduction:** 0.843 for all variables, 0.802 for the importance variables, and 0.799 on the EDA features
- **linear dim reduction**
  - all variables: 0.758 for PCA, 0.707 for SVD, 0.775 for KMeans
  - importance variables: 0.759 for PCA, 0.746 for SVD, 0.783 for KMeans
  - EDA variables: 0.782 for PCA, 0.79 for SVD, 0.783 for KMeans
- **non-linear dim reduction (KPCA):** 0.745 for all variables, 0.755 for importance variables, 0.765 for EDA variables
- **linear + non-linear combinations**
  - all variables: 0.734 for PCA + KPCA, 0.698 for SVD + KPCA, 0.789 for KMeans + KPCA
  - importance variables: 0.756 for PCA + KPCA, 0.753 for SVD + KPCA, 0.791 for KMeans + KPCA
  - EDA variables: not computed due to limited time and computational resources and considered performances of previous feature selections

  <br>

As you can see, the best results were provided by **not doing any dimension reduction, and using all variables**.

As a final attempt to improve my performance in the competition, I submitted three of the models used in the ensemble, with the **final preprocessing pipeline** (oversampling, removing certain missing data, Iterative Imputer, no scaling). I tuned the hyperparameters of the estimators AdaBoost, Gradient Boosting and Random Forest, for 25 Bayes Search iterations.

Unfortunately, these models were not able to improve my previous score in the competition.

Therefore, the **final winning model** is the **ensemble of the 11 default estimators with soft-voting**, which achieved the following performances:

**test score** of **0.829**

**public score** of **0.85894**

**private score** of **0.88140**

## Conclusions

I was quite impressed with my final ranking in the leaderboard, specially taking into account that I **moved up 15 places** (from 17th on the public leaderboard).

This made me question how was this possible.

After researching into how the public and private scores are computed (if test set consists of 100 rows, the public score is the best score you get on a certain percentage of these rows, and the private score is the score you get on the rest of the rows, **using the same best performing model on the public dataset**).

Then, I looked at the public and private leaderboards, and I realized that most of the top ranks in the public score had many submissions, but on the private leaderboard ended up performing much worse.

This was likely because by submitting as many competitive models as possible, effectively they were **overfitting the public dataset**, thus resulting in a worse performance on the private dataset.

<br>

This event also speaks to the fact that the most **reliable strategy** is to **base your results on your own test** dataset (from the training set). A reasonable strategy would thus be to take a small number of models and preprocesing pipelines that are able to generally improve the performance on this test set, and evaluate if these models are also able to improve the performance in the competition.  
If this is the case, you know that your train and test split is comparable to the overall training set you have, and the unkown target variable in the competition.

Therefore, the number of model to submit should be enough so that you achieve a **reasonably competitive score** on **your test set**, and not a very large number of submissions.

This way is the most reliable strategy to **train a model that is able to generalize**, which is ultimately the objective of the competition, since the final **private score is completely unkown**.

<br>

I think with this in mind, it makes more sense that my model did so well.

Of course there was some luck involved, but I did analyze the best preprocessing pipelines **on my own test set** and decided that the best preprocessing pipeline was to use **all features** and **no dimensionality reduction**, which is what my best model had.

Therefore, in reality I was implementing the most **reliable preprocessing pipeline**, even though I did it beforehand.